<a href="https://colab.research.google.com/github/alexkrish/Projection-mehod-for-solving-linear-operator-equations/blob/main/Projection_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projection methods for solving linear operator equations

# Creating (in)finite system of eq due to task

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def generate_b1(n, add=False):
    if not add:
        b = 0.5 * np.array([1 / (2 * i)**j for i in range(1, n+1) for j in range(1, n+1)])
        return b.reshape(n, n)
    else:
        return (0.5 * np.array([1 / (2 * i)** n for i in range(1, n)]), 0.5 * np.array([1 / (2 * n)** j for j in range(1, n+1)]))


def generate_g1(n, add=False):
    if not add:
        return np.array([ 1 / (2**(i-1)) for i in range(1, n+1)])
    else:
        return 1 / 2**(n-1)

In [ ]:
def generate_b2(n, add=False):
    if not add:
        b = 0.5 * np.array([1 / 2**(i+j) for i in range(1, n+1) for j in range(1, n+1)])
        return b.reshape(n, n)
    else:
        return (0.5 * np.array([1 / 2**(i+n) for i in range(1, n)]), 0.5 * np.array([1 / 2**(n+j) for j in range(1, n+1)]))


def generate_g2(n, add=False):
    if not add:
        return np.array([ 1 / (2**(i)) for i in range(1, n+1)])
    else:
        return 1 / 2**(n)

In [ ]:
class ProjectionMethod:
    """
        Class Projection Method solves inf system of linear alg equations using 
        projection method, reduction method. Comparing nearest vectors depends
        on space. This class provides 2 types of spaces: l2 and m, which can be chosen 
        in main function.
    """
    def __init__(self, b, g, N=4, eps=1e-5):
        self.b = b  # function that define matrix of coef near variables
        self.g = g  # function that define free coef
        self.N = N  # order of first equation from wich we about to start
        self.final_n = N  # last system order where calculations stopped
        self.eps = eps  # precission
        self.x = []  # array of solutions
        self.norma = []  # array of norm values
        self.residual = []  # array of residual vector
        self.determinant = []  # array of det values
        self.space = ''  # type of space that is used
    
    def projection_method(self, space='l2'):
        """
            Main method that solves problem of finding roots in inf system
        """
        n = self.N
        self.space = space
        if space == 'l2':
            # if space l2 than norm is euclidean and elements starting from 
            # (n + 1) complemented by 0.0
            norm = self.euclidean_norm
            adder = lambda vec: 0.0
        elif space == 'm':
            # if space is m than norm is first vector norm and elements starting
            # from (n + 1) complemented by mean of first n
            norm = self.first_norm
            adder = lambda vec: np.sum(vec) / len(vec)

        # creates systems with n and n-1 order
        B, G = self.system(n), self.g(n)
        B_perv, G_perv = self.system(n-1), self.g(n-1)
        # checking determinant and adding to table
        self.determinant.append(self.det(B_perv))
        # find roots of n-1 order system
        self.x.append(self.ortogonalization(B_perv, G_perv))
        # and complement it to n order
        self.x[-1] = np.append(self.x[-1], adder(self.x[-1]))
        # solve n order system
        self.x.append(self.ortogonalization(B, G))
        # and add to table residual vector (g - b * x)
        self.residual.append(self.residual_vec(n, self.x[-1]))
        
        # init criteria that will stop process.
        # This is used in case it is impossible to satisfy first stop criteria
        # in finite amount of iterations
        stop_criteria = 0.05 * norm(self.x[1] - self.x[0])

        while norm(self.x[-1] - self.x[-2]) > self.eps:
            if n >23:
                if norm(self.x[-1] - self.x[-2]) < stop_criteria:
                    break
            # checking 2 stop criterias. But second should be applied after
            # some iterations 

            # append norm to the table
            self.norma.append(norm(self.x[-1] - self.x[-2]))

            # increase order of system
            n += 1

            # updating already existing systems
            B, G = self.update_system(B, G, n)

            # complemnt last solution to new n ordet. It will allow to compare systems
            self.x[-1] = np.append(self.x[-1], adder(self.x[-1]))

            # append value of det to table
            self.determinant.append(self.det(B))

            # find and append new roots to table
            self.x.append(self.ortogonalization(B, G))

            # append residual vector to table
            self.residual.append(self.residual_vec(n, self.x[-1]))

            # for debug purpose
            # ----------------
            print('n = ', n)
            print('norm = ', norm(self.x[-1] - self.x[-2]))
            print('stop_criteria = ', stop_criteria)
            print(norm(self.x[-1] - self.x[-2]) > stop_criteria)
            # ---------------

        # get max order of system
        self.final_n = n

        # return last x
        return self.x[-1]
    
    def generate_diag_matr(self, n, add=False):
        """
            Function that generates diagonal matrix n x n in case add=False.
            In case add = True - generates array size of n with 1 in n coordinate.
            THis was made to merge only one row to already existing system to
            minimize computation time 
        """
        if not add:
            return np.eye(n, n)
        else:
            tmp = np.zeros(n)
            tmp[-1] = 1
            return tmp

    def system(self, n, add=False):
        """
            Function that generates system size of n x n in case add = False.
            Made by rule: substr matrix b(n) from diagonal matrix.
            In case add = True - generates 2 arrays. one of them is column size of n-1
            and second is row size of n.
        """
        if not add:
            return self.generate_diag_matr(n) - self.b(n)
        else:
            return -self.b(n, add=True)[0], self.generate_diag_matr(n, add=True) - self.b(n, add=True)[1]
    
    def update_system(self, matrix, vector, n):
        """
            Function that updates already existing system. Due to this system
            and free coef transform from n-order to (n+1)-order
        """
        col, row = self.system(n, add=True)
        matrix = np.column_stack((matrix, col))
        matrix = np.vstack([matrix, row])

        tmp = self.g(n, add=True)
        vector = np.append(vector, tmp)

        return matrix, vector

    def ortogonalization(self, A, f):
        """Ortogonalization Method that solves finite system of lin alg eq."""
        n = len(f)

        # firstly concatenate free coef to system
        f = np.reshape((f * -1), (n, 1))
        a = np.concatenate((A, f), axis=1)

        # complement system by vector that equals 1  
        a = np.concatenate((a, [[0] * n + [1]]), axis=0)
        b = np.zeros((n + 1, n + 1))

        # building ortonorm system
        b[0] = a[0] / np.linalg.norm(a[0])
        for i in range(1, n + 1):
            u = a[i] - np.sum([np.dot(a[i], b[j]) * b[j] for j in range(i)], axis=0)
            b[i] = u / np.linalg.norm(u)
        
        # find x from ortonormalized system
        x = [b[-1][i] / b[-1][-1] for i in range(n)]
        return np.array(x)
    
    def first_norm(self, vec):
        """Function that returns first vector norm for vec"""
        return np.sum(np.abs(vec))

    def euclidean_norm(self, vec):
        """Function that returns euclidean norm for vec"""
        return np.sqrt(np.sum(vec**2))

    def det(self, matrix):
        """
            Function that checks determinant of matrix.
            In case det = 0 raise ValueError that 'Determinant is null'
            Else returns value
        """
        assert np.linalg.det(matrix) != 0, 'Determinant is null'
        return np.linalg.det(matrix)
    
    def stats(self):
        """
            Function that creates pandas DataFrame with statistics of projection method process.
        """
        adder = {'n': '',
                 'xn': '',
                 '||xn - xn-1||': '',
                 'residual': '', 
                 'det': ''}
        df = pd.DataFrame(columns=list(adder.keys()))

        for i in range(self.final_n - self.N):
            adder['n'] = i + self.N
            adder['xn'] = self.x[i]
            adder['||xn - xn-1||'] = self.norma[i]
            adder['residual'] = self.residual[i]
            adder['det'] = self.determinant[i]

            df = df.append(adder, ignore_index=True)
        return df
    
    def residual_vec(self, n, x):
        """ Function that return residual vector for x"""
        return self.g(n) - self.system(n) @ x

# Tests

*$X = l_{2}$*  
*$x_{i} - 0.5 \sum\limits_{j=1}^{\inf} \frac {x_{j}} {2i^j}  = \frac {1}{2^{i-1}},   i = 1,2,...$*

<iframe src="https://drive.google.com/file/d/1q7BCH57rVMHXXfOputQgIw970ZMewbzW/preview" width="640" height="480" allow="autoplay"></iframe>


In [ ]:
pm = ProjectionMethod(generate_b1, generate_g1)
x1 = pm.projection_method()

n =  5
norm =  0.14113179271536289
stop_criteria =  0.011226651357151849
True
n =  6
norm =  0.09624931453741145
stop_criteria =  0.011226651357151849
True
n =  7
norm =  0.07101824367923802
stop_criteria =  0.011226651357151849
True
n =  8
norm =  0.056071360194169646
stop_criteria =  0.011226651357151849
True
n =  9
norm =  0.04665781330228487
stop_criteria =  0.011226651357151849
True
n =  10
norm =  0.040325216203580656
stop_criteria =  0.011226651357151849
True
n =  11
norm =  0.035782914311665526
stop_criteria =  0.011226651357151849
True
n =  12
norm =  0.03233518572907826
stop_criteria =  0.011226651357151849
True
n =  13
norm =  0.029595403350841182
stop_criteria =  0.011226651357151849
True
n =  14
norm =  0.02734040075700926
stop_criteria =  0.011226651357151849
True
n =  15
norm =  0.025435431692627936
stop_criteria =  0.011226651357151849
True
n =  16
norm =  0.023794970803550794
stop_criteria =  0.011226651357151849
True
n =  17
norm =  0.022361877757534038
stop_criteria 

In [ ]:
pm.stats()

,n,xn,||xn - xn-1||,residual,det
0,4,"[1.4838158604567528, 0.7106892388316595, 0.384...",0.224533,"[0.0, 4.440892098500626e-16, -2.77555756156289...",0.703975
1,5,"[1.4935244957800566, 0.7124016446910103, 0.385...",0.141132,"[4.440892098500626e-16, -3.3306690738754696e-1...",0.701166
2,6,"[1.496572577408649, 0.7128686392729524, 0.3856...",0.096249,"[-4.440892098500626e-16, -6.661338147750939e-1...",0.700847
3,7,"[1.4976109631639176, 0.7130156368384251, 0.385...",0.071018,"[1.1102230246251565e-16, -3.3306690738754696e-...",0.700710
4,8,"[1.497993830928669, 0.7130676021556361, 0.3857...",0.056071,"[3.3306690738754696e-16, 4.440892098500626e-16...",0.700651
5,9,"[1.4981449256536206, 0.7130876682607641, 0.385...",0.046658,"[-6.661338147750939e-16, -1.1102230246251565e-...",0.700624
6,10,"[1.498207778829265, 0.7130959235984229, 0.3857...",0.040325,"[1.7763568394002505e-15, -2.220446049250313e-1...",0.700613
7,11,"[1.4982349375926978, 0.7130994708657048, 0.385...",0.035783,"[-2.220446049250313e-16, 0.0, 1.11022302462515...",0.700607
8,12,"[1.4982469868001649, 0.713101040235761, 0.3857...",0.032335,"[0.0, 2.220446049250313e-16, 2.775557561562891...",0.700605
9,13,"[1.4982524307933967, 0.713101748303223, 0.3857...",0.029595,"[9.992007221626409e-16, 5.551115123125783e-16,...",0.700604


*$X = l_{2}$*  
*$x_{i} - 0.5 \sum\limits_{j=1}^{\inf} \frac {x_{j}} {2^{i+j}}  = \frac {1}{2^{i}},   i = 1,2,...$*

In [ ]:
pm = ProjectionMethod(generate_b2, generate_g2)
x2 = pm.projection_method()

n =  5
norm =  0.03749486198784293
stop_criteria =  0.0037479316211262654
True
n =  6
norm =  0.018749358784789477
stop_criteria =  0.0037479316211262654
True
n =  7
norm =  0.009374919880539017
stop_criteria =  0.0037479316211262654
True
n =  8
norm =  0.004687489986081574
stop_criteria =  0.0037479316211262654
True
n =  9
norm =  0.0023437487482918925
stop_criteria =  0.0037479316211262654
False
n =  10
norm =  0.0011718748435374771
stop_criteria =  0.0037479316211262654
False
n =  11
norm =  0.0005859374804422155
stop_criteria =  0.0037479316211262654
False
n =  12
norm =  0.0002929687475552779
stop_criteria =  0.0037479316211262654
False
n =  13
norm =  0.00014648437469440983
stop_criteria =  0.0037479316211262654
False
n =  14
norm =  7.32421874618012e-05
stop_criteria =  0.0037479316211262654
False
n =  15
norm =  3.662109374522516e-05
stop_criteria =  0.0037479316211262654
False
n =  16
norm =  1.8310546874403146e-05
stop_criteria =  0.0037479316211262654
False
n =  17
norm =  9

In [ ]:
pm.stats()

,n,xn,||xn - xn-1||,residual,det
0,4,"[0.5981308411214954, 0.2990654205607476, 0.149...",0.074959,"[2.220446049250313e-16, -5.551115123125783e-17...",0.835938
1,5,"[0.5995316159250583, 0.29976580796252933, 0.14...",0.037495,"[-1.1102230246251565e-16, -5.551115123125783e-...",0.833496
2,6,"[0.5998828353837142, 0.2999414176918571, 0.149...",0.018749,"[5.551115123125783e-17, 8.326672684688674e-17,...",0.833374
3,7,"[0.5999707045554415, 0.2999853522777208, 0.149...",0.009375,"[2.220446049250313e-16, 0.0, 0.0, 0.0, -6.9388...",0.833344
4,8,"[0.5999926758706556, 0.29999633793532793, 0.14...",0.004687,"[-2.220446049250313e-16, -1.1102230246251565e-...",0.833336
5,9,"[0.5999981689509006, 0.2999990844754503, 0.149...",0.002344,"[0.0, 1.6653345369377348e-16, -2.7755575615628...",0.833334
6,10,"[0.5999995422366774, 0.29999977111833853, 0.14...",0.001172,"[-1.1102230246251565e-16, 2.7755575615628914e-...",0.833333
7,11,"[0.5999998855591039, 0.2999999427795519, 0.149...",0.000586,"[5.551115123125783e-17, 8.326672684688674e-17,...",0.833333
8,12,"[0.5999999713897718, 0.29999998569488584, 0.14...",0.000293,"[0.0, 1.3877787807814457e-16, 0.0, 3.469446951...",0.833333
9,13,"[0.5999999928474428, 0.2999999964237212, 0.149...",0.000146,"[5.551115123125783e-17, 1.3877787807814457e-16...",0.833333


*$X = m$*  
*$x_{i} - 0.5 \sum\limits_{j=1}^{\inf} \frac {x_{j}} {2i^j}  = \frac {1}{2^{i-1}},   i = 1,2,...$*

In [ ]:
pm = ProjectionMethod(generate_b1, generate_g1)
x3 = pm.projection_method(space='m')

n =  5
norm =  0.5667777581334446
stop_criteria =  0.03238334655334487
True
n =  6
norm =  0.4972815314390281
stop_criteria =  0.03238334655334487
True
n =  7
norm =  0.4392257159280704
stop_criteria =  0.03238334655334487
True
n =  8
norm =  0.39126151189504743
stop_criteria =  0.03238334655334487
True
n =  9
norm =  0.3516978788605571
stop_criteria =  0.03238334655334487
True
n =  10
norm =  0.31892233356463817
stop_criteria =  0.03238334655334487
True
n =  11
norm =  0.2915584860725387
stop_criteria =  0.03238334655334487
True
n =  12
norm =  0.2684944296354683
stop_criteria =  0.03238334655334487
True
n =  13
norm =  0.24885662814186255
stop_criteria =  0.03238334655334487
True
n =  14
norm =  0.2319673645562466
stop_criteria =  0.03238334655334487
True
n =  15
norm =  0.2173025440315452
stop_criteria =  0.03238334655334487
True
n =  16
norm =  0.2044558368372646
stop_criteria =  0.03238334655334487
True
n =  17
norm =  0.1931102806222187
stop_criteria =  0.03238334655334487
True
n

In [ ]:
pm.stats()

,n,xn,||xn - xn-1||,residual,det
0,4,"[1.4838158604567528, 0.7106892388316595, 0.384...",0.647667,"[0.0, 4.440892098500626e-16, -2.77555756156289...",0.703975
1,5,"[1.4935244957800566, 0.7124016446910103, 0.385...",0.566778,"[4.440892098500626e-16, -3.3306690738754696e-1...",0.701166
2,6,"[1.496572577408649, 0.7128686392729524, 0.3856...",0.497282,"[-4.440892098500626e-16, -6.661338147750939e-1...",0.700847
3,7,"[1.4976109631639176, 0.7130156368384251, 0.385...",0.439226,"[1.1102230246251565e-16, -3.3306690738754696e-...",0.700710
4,8,"[1.497993830928669, 0.7130676021556361, 0.3857...",0.391262,"[3.3306690738754696e-16, 4.440892098500626e-16...",0.700651
...,...,...,...,...,...
110,114,"[1.4982570817636123, 0.7131023526731444, 0.385...",0.033650,"[1.4432899320127035e-15, 7.216449660063518e-16...",0.700603
111,115,"[1.498257081763612, 0.7131023526731441, 0.3857...",0.033384,"[1.887379141862766e-15, 6.661338147750939e-16,...",0.700603
112,116,"[1.4982570817636116, 0.7131023526731441, 0.385...",0.033121,"[1.887379141862766e-15, 8.881784197001252e-16,...",0.700603
113,117,"[1.4982570817636116, 0.7131023526731439, 0.385...",0.032864,"[2.1094237467877974e-15, 9.992007221626409e-16...",0.700603


*$X = m$*  
*$x_{i} - 0.5 \sum\limits_{j=1}^{\inf} \frac {x_{j}} {2^{i+j}}  = \frac {1}{2^{i}},   i = 1,2,...$*

In [ ]:
pm = ProjectionMethod(generate_b2, generate_g2)
x4 = pm.projection_method(space='m')

n =  5
norm =  0.2441963042383686
stop_criteria =  0.013820978061823775
True
n =  6
norm =  0.21387576071405337
stop_criteria =  0.013820978061823775
True
n =  7
norm =  0.18753375789860327
stop_criteria =  0.013820978061823775
True
n =  8
norm =  0.16541062409058663
stop_criteria =  0.013820978061823775
True
n =  9
norm =  0.14707259415675533
stop_criteria =  0.013820978061823775
True
n =  10
norm =  0.13190162566743302
stop_criteria =  0.013820978061823775
True
n =  11
norm =  0.11929702365580995
stop_criteria =  0.013820978061823775
True
n =  12
norm =  0.10874471099401434
stop_criteria =  0.013820978061823775
True
n =  13
norm =  0.09982911109735133
stop_criteria =  0.013820978061823775
True
n =  14
norm =  0.09222318449821709
stop_criteria =  0.013820978061823775
True
n =  15
norm =  0.08567243364237455
stop_criteria =  0.013820978061823775
True
n =  16
norm =  0.07997924819960799
stop_criteria =  0.013820978061823775
True
n =  17
norm =  0.07498970035579937
stop_criteria =  0.013

In [ ]:
pm.stats()

,n,xn,||xn - xn-1||,residual,det
0,4,"[0.5981308411214954, 0.2990654205607476, 0.149...",0.276420,"[2.220446049250313e-16, -5.551115123125783e-17...",0.835938
1,5,"[0.5995316159250583, 0.29976580796252933, 0.14...",0.244196,"[-1.1102230246251565e-16, -5.551115123125783e-...",0.833496
2,6,"[0.5998828353837142, 0.2999414176918571, 0.149...",0.213876,"[5.551115123125783e-17, 8.326672684688674e-17,...",0.833374
3,7,"[0.5999707045554415, 0.2999853522777208, 0.149...",0.187534,"[2.220446049250313e-16, 0.0, 0.0, 0.0, -6.9388...",0.833344
4,8,"[0.5999926758706556, 0.29999633793532793, 0.14...",0.165411,"[-2.220446049250313e-16, -1.1102230246251565e-...",0.833336
...,...,...,...,...,...
79,83,"[0.5999999999999999, 0.29999999999999993, 0.14...",0.014634,"[-2.220446049250313e-16, 2.220446049250313e-16...",0.833333
80,84,"[0.5999999999999999, 0.29999999999999993, 0.14...",0.014458,"[-2.220446049250313e-16, 2.220446049250313e-16...",0.833333
81,85,"[0.5999999999999999, 0.29999999999999993, 0.14...",0.014286,"[-2.220446049250313e-16, 2.220446049250313e-16...",0.833333
82,86,"[0.5999999999999999, 0.29999999999999993, 0.14...",0.014118,"[-2.220446049250313e-16, 2.220446049250313e-16...",0.833333
